In [108]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *

In [109]:
spark = SparkSession.builder.master('local[*]').appName('Exercise2').getOrCreate()

In [110]:
df_prod = spark.read.parquet('../data/products_parquet', header=True, inferSchema=True)

In [111]:
df_prod.show()

+----------+------------+-----+
|product_id|product_name|price|
+----------+------------+-----+
|         0|   product_0|   22|
|         1|   product_1|   30|
|         2|   product_2|   91|
|         3|   product_3|   37|
|         4|   product_4|  145|
|         5|   product_5|  128|
|         6|   product_6|   66|
|         7|   product_7|  145|
|         8|   product_8|   51|
|         9|   product_9|   44|
|        10|  product_10|   53|
|        11|  product_11|   13|
|        12|  product_12|  104|
|        13|  product_13|  102|
|        14|  product_14|   24|
|        15|  product_15|   14|
|        16|  product_16|   38|
|        17|  product_17|   72|
|        18|  product_18|   16|
|        19|  product_19|   46|
+----------+------------+-----+
only showing top 20 rows



In [112]:
df_sales = spark.read.parquet('../data/sales_parquet', header=True, inferSchema=True)

In [113]:
df_sales.show()

+--------+----------+---------+----------+---------------+--------------------+
|order_id|product_id|seller_id|      date|num_pieces_sold|       bill_raw_text|
+--------+----------+---------+----------+---------------+--------------------+
|       1|         0|        0|2020-07-10|             62|esctmitgntlqljxnC...|
|       2|         0|        0|2020-07-07|             38|cufiduzyskiviokju...|
|       3|         0|        0|2020-07-05|             13|hscngebsortzolelf...|
|       4|         0|        0|2020-07-06|             94|jxhvzoobncxwzkpdl...|
|       5|         0|        0|2020-07-02|             41|nqazvvrqffccuwzpr...|
|       6|         0|        0|2020-07-07|             72|auesyqwlzglbecnmn...|
|       7|         0|        0|2020-07-04|             58|wymwvtmlsrirflpne...|
|       8|         0|        0|2020-07-08|             97|xzsadhvwyzhiboqIu...|
|       9|         0|        0|2020-07-08|             53|gvbzspbwezmfjwmuz...|
|      10|         0|        0|2020-07-0

In [114]:
df_sales.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- num_pieces_sold: string (nullable = true)
 |-- bill_raw_text: string (nullable = true)



In [115]:
df_sellers = spark.read.parquet('../data/sellers_parquet', header=True, inferSchema=True)

In [116]:
df_sellers.show()

+---------+-----------+------------+
|seller_id|seller_name|daily_target|
+---------+-----------+------------+
|        0|   seller_0|        2500|
|        1|   seller_1|      988094|
|        2|   seller_2|      117965|
|        3|   seller_3|     1157798|
|        4|   seller_4|     1999213|
|        5|   seller_5|     1793991|
|        6|   seller_6|      499726|
|        7|   seller_7|      888438|
|        8|   seller_8|     1528179|
|        9|   seller_9|      442654|
+---------+-----------+------------+



**1.Find out how many orders, how many products and how many sellers are in the data.**

In [117]:
df_prod.count()

75000

In [118]:
df_sales.count()

200040

In [119]:
df_sellers.count()

10

**2.How many products have been sold at least once? Which is the product contained in more orders?**

In [120]:
df_sales.select('product_id').distinct().count()

9413

In [121]:
# df_sales.groupby('product_id').agg(count('order_id').alias('cnt')).sort(desc('cnt')).show(1)
df_sales.createOrReplaceTempView('Sales')
spark.sql('Select product_id, count(order_id) cnt from Sales group by product_id order by cnt desc limit 1').show()

+----------+------+
|product_id|   cnt|
+----------+------+
|         0|190000|
+----------+------+



**3.How many distinct products have been sold in each day?**

In [122]:
df_sales.groupby('date').agg(count('product_id').alias('cnt')).show()

+----------+-----+
|      date|  cnt|
+----------+-----+
|2020-07-03|20107|
|2020-07-07|20042|
|2020-07-01|19992|
|2020-07-08|20042|
|2020-07-04|20055|
|2020-07-10|19725|
|2020-07-09|19930|
|2020-07-06|20122|
|2020-07-02|20023|
|2020-07-05|20002|
+----------+-----+



**4.What is the average revenue of the orders?**

In [123]:
df_sales_det = df_sales.join(df_prod, 'product_id', 'inner').withColumn('product_revenue', col('num_pieces_sold') * col('price'))

In [124]:
df_rev = df_sales_det.groupby().agg({'product_revenue': 'sum', '*': 'count'})

In [125]:
df_rev.withColumn('avg_rev', col('sum(product_revenue)') / col('count(1)')).select('avg_rev').show()

+------------------+
|           avg_rev|
+------------------+
|1249.0325584883024|
+------------------+



**5.For each seller, what is the average % contribution of an order to the seller's daily quota?**

In [126]:
df_sales_det = df_sales.join(df_sellers, 'seller_id', 'inner').withColumn('prod_quota', col('num_pieces_sold') / col('daily_target'))

In [127]:
df_sales_det.groupby().agg({'prod_quota':'sum', '*':'count'}).withColumn('avg_contrib', col('sum(prod_quota)') / col('count(1)')).select('avg_contrib').show()

+--------------------+
|         avg_contrib|
+--------------------+
|0.019218458886404645|
+--------------------+



**6.Who are the second most selling and the least selling persons (sellers) for each product? Who are those for product with `product_id = 0`**

In [128]:
window = Window.partitionBy('product_id').orderBy(col('pieces_per_product').desc())
df_sales.groupby('seller_id', 'product_id').agg(sum('num_pieces_sold').alias('pieces_per_product')) \
    .withColumn('row_num', row_number().over(window)).filter(col('row_num') == 2).orderBy('product_id', 'seller_id').show()

+---------+----------+------------------+-------+
|seller_id|product_id|pieces_per_product|row_num|
+---------+----------+------------------+-------+
|        8|     10013|              51.0|      2|
|        1|     10169|              41.0|      2|
|        8|     10308|               7.0|      2|
|        3|     10386|              36.0|      2|
|        6|     10454|              64.0|      2|
|        7|     10618|               7.0|      2|
|        6|     10866|              50.0|      2|
|        2|      1093|              75.0|      2|
|        6|     11212|              31.0|      2|
|        5|     11466|              15.0|      2|
|        6|     11700|              43.0|      2|
|        9|     11704|              53.0|      2|
|        7|     11779|              17.0|      2|
|        6|     11833|              43.0|      2|
|        6|      1184|              59.0|      2|
|        9|      1199|              64.0|      2|
|        9|     12078|               9.0|      2|


In [129]:
window = Window.partitionBy('product_id').orderBy(col('pieces_per_product'))
df_ls = df_sales.groupby('seller_id', 'product_id').agg(sum('num_pieces_sold').alias('pieces_per_product')) \
    .withColumn('row_num', row_number().over(window))
df_ls.filter((col('row_num') >= 1)).orderBy('product_id', 'seller_id').show()

+---------+----------+------------------+-------+
|seller_id|product_id|pieces_per_product|row_num|
+---------+----------+------------------+-------+
|        0|         0|         9608720.0|      1|
|        6|     10001|              51.0|      1|
|        4|     10005|              53.0|      1|
|        3|     10011|               4.0|      1|
|        2|     10013|              81.0|      2|
|        8|     10013|              51.0|      1|
|        8|     10017|              97.0|      1|
|        4|     10048|              26.0|      1|
|        3|     10054|              79.0|      1|
|        5|     10062|              89.0|      1|
|        7|     10072|              39.0|      1|
|        2|     10076|              71.0|      1|
|        2|      1008|              68.0|      1|
|        8|     10095|              41.0|      1|
|        9|     10096|              64.0|      1|
|        9|     10099|              77.0|      1|
|        9|     10100|              97.0|      1|


In [130]:
window = Window.partitionBy('product_id').orderBy('pieces_per_product')
df_sales.groupby('seller_id', 'product_id').agg(sum('num_pieces_sold').alias('pieces_per_product')) \
    .withColumn('row_num', row_number().over(window)).filter(col('product_id') == '0').show()

+---------+----------+------------------+-------+
|seller_id|product_id|pieces_per_product|row_num|
+---------+----------+------------------+-------+
|        0|         0|         9608720.0|      1|
+---------+----------+------------------+-------+



**7.Create a new column called "hashed_bill" defined as follows:**
- if the order_id is even: apply MD5 hashing iteratively to the bill_raw_text field, once for each 'A' (capital 'A') present in the text. E.g. if the bill text is 'nbAAnllA', you would apply hashing three times iteratively (only if the order number is even)
- if the order_id is odd: apply SHA256 hashing to the bill text
- check if there are any duplicate on the new column

In [131]:
import hashlib

def hashing(order_id, brt):
    if int(order_id) % 2 == 0:
        cnt = brt.count('A')
        md5str = brt
        for i in range(cnt):
            md5str = hashlib.md5(md5str.encode('utf-8')).hexdigest()
        return md5str
    else:
        return hashlib.sha256(brt.encode('utf-8')).hexdigest()
    
# print(hashing(5, 'AsctmitgntlqljxnCAA'))

apply_hashing = udf(hashing)



In [132]:
df_hash = df_sales.withColumn('hashed_bill', apply_hashing('order_id', 'bill_raw_text'))

In [133]:
df_hash.createOrReplaceTempView('Hashed_Sales')
spark.sql('Select hashed_bill, count(*) cnt from Hashed_Sales group by hashed_bill having count(*) > 1').show()

+-----------+---+
|hashed_bill|cnt|
+-----------+---+
+-----------+---+

